In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset
from tqdm import tqdm
import random
import os

In [3]:
device = 'cuda'

In [4]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
seed_everything(42)

# Dataset

In [6]:
df_train = pd.read_csv('./data/train_result_mean_median_max_min_current_next_prev.csv')
df_test = pd.read_csv('./data/test_result_mean_median_max_min_current_next_prev.csv')

In [7]:
df_train.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_next_min,X4_next_min,X2_prev_min,X4_prev_min,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,780,905,867,931,954,935,40,1,2,7
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2110,2096,2224,2126,2026,2034,17,13,8,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1822,2153,2092,2047,1883,2053,1,1,2,4
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1416,1506,1564,1502,1474,1,1,1,26
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1263,1264,1239,1162,1202,1191,8,10,19,1


In [8]:
binarizer = MultiLabelBinarizer(classes=np.arange(0,101))

In [9]:
fig_1_train = binarizer.fit_transform(df_train.iloc[:, 6:14].as_matrix())
fig_2_train = binarizer.transform(df_train.iloc[:, 14:22].as_matrix())
fig_1_test = binarizer.transform(df_test.iloc[:, 6:14].as_matrix())
fig_2_test = binarizer.transform(df_test.iloc[:, 14:22].as_matrix())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [10]:
df_train['1/to_next_game_1'] = df_train['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_next_game_2'] = df_train['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_1'] = df_train['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_2'] = df_train['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

df_test['1/to_next_game_1'] = df_test['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_next_game_2'] = df_test['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_1'] = df_test['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_2'] = df_test['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

In [11]:
df_train

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2,1/to_next_game_1,1/to_next_game_2,1/to_prev_game_1,1/to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,954,935,40,1,2,7,0.025000,1.000000,0.500000,0.142857
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2026,2034,17,13,8,1,0.058824,0.076923,0.125000,1.000000
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1883,2053,1,1,2,4,1.000000,1.000000,0.500000,0.250000
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1474,1,1,1,26,1.000000,1.000000,1.000000,0.038462
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1202,1191,8,10,19,1,0.125000,0.100000,0.052632,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1211,1049,6,0,16,166,0.166667,0.000000,0.062500,0.006024
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,699,706,0,18,4,10,0.000000,0.055556,0.250000,0.100000
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1063,1036,5,1,257,0,0.200000,1.000000,0.003891,0.000000
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1752,1719,0,1,4,1,0.000000,1.000000,0.250000,1.000000


In [12]:
rating_1_train = df_train[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_train = df_train[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

rating_1_test =  df_test[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1']].values
rating_2_test = df_test[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2']].values

In [13]:
y_train = df_train['target']
rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train = shuffle(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42)

In [14]:
rating_1_train, rating_1_val, \
rating_2_train, rating_2_val, \
fig_1_train, fig_1_val, \
fig_2_train, fig_2_val, \
y_train, y_val = train_test_split(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42, test_size=0.2)

In [15]:
# figs_train = np.concatenate([fig_1_train, fig_2_train], axis=1)
# figs_val = np.concatenate([fig_1_val, fig_2_val], axis=1)
# figs_test = np.concatenate([fig_1_test, fig_2_test], axis=1)

# rating_train = np.concatenate([rating_1_train, rating_2_train], axis=1)
# rating_val = np.concatenate([rating_1_val, rating_2_val], axis=1)
# rating_test = np.concatenate([rating_1_test, rating_2_test], axis=1)





# Data for torch

In [16]:
class TrainDataset(Dataset):
    def __init__(self, figs_1, figs_2, rating_1, rating_2, y):
        self.figs_1 = torch.tensor(figs_1, dtype=torch.float)
        self.figs_2 = torch.tensor(figs_2, dtype=torch.float)
        self.rating_1 = torch.tensor(rating_1, dtype=torch.float)
        self.rating_2 = torch.tensor(rating_2, dtype=torch.float)
        self.y = torch.tensor(y.tolist(), dtype=torch.float)
    
    def __len__(self):
        return len(self.figs_1)
    
    def __getitem__(self, idx):
        # можно добавить случайный своп
        return {
            'figs_1': self.figs_1[idx],
            'figs_2': self.figs_2[idx],
            'rating_1': self.rating_1[idx],
            'rating_2': self.rating_2[idx],
            'y': self.y[idx]
        }

class TestDataset(Dataset):
    def __init__(self,  figs_1, figs_2, rating_1, rating_2):
        self.figs_1 = torch.tensor(figs_1, dtype=torch.float)
        self.figs_2 = torch.tensor(figs_2, dtype=torch.float)
        self.rating_1 = torch.tensor(rating_1, dtype=torch.float)
        self.rating_2 = torch.tensor(rating_2, dtype=torch.float)
    
    def __len__(self):
        return len(self.figs_1)
    
    def __getitem__(self, idx):
        return {
            'figs_1': self.figs_1[idx],
            'figs_2': self.figs_2[idx],
            'rating_1': self.rating_1[idx],
            'rating_2': self.rating_2[idx],
        }
        

In [17]:
def train_epoch(iterator, optimizer, criterion):
        model.train()
        epoch_loss = 0

        for i, batch in tqdm(enumerate(iterator)):
            targets = batch['y'].to(device)
            figs_1 = batch['figs_1'].to(device)
            figs_2 = batch['figs_2'].to(device)
            rating_1 = batch['rating_1'].to(device)
            rating_2 = batch['rating_2'].to(device)

            optimizer.zero_grad()

            logits = model(figs_1, figs_2, rating_1, rating_2).squeeze()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def val_epoch(iterator,criterion):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                targets = batch['y'].to(device)
                figs_1 = batch['figs_1'].to(device)
                figs_2 = batch['figs_2'].to(device)
                rating_1 = batch['rating_1'].to(device)
                rating_2 = batch['rating_2'].to(device)
    
                logits = model(figs_1, figs_2, rating_1, rating_2).squeeze()
                loss = criterion(logits, targets)
                epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def predict(iterator):
        model.eval()
        preds = []
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                figs_1 = batch['figs_1'].to(device)
                figs_2 = batch['figs_2'].to(device)
                rating_1 = batch['rating_1'].to(device)
                rating_2 = batch['rating_2'].to(device)
                pred = model.predict(figs_1, figs_2, rating_1, rating_2)
                preds += pred.to('cpu').numpy().tolist()

        return preds

In [18]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1figs = nn.Linear(101, 128)
        self.bn1figs = nn.BatchNorm1d(128)        
        self.fc2figs = nn.Linear(128, 32)
        self.bn2figs = nn.BatchNorm1d(32)
       
        self.fc1rating = nn.Linear(17, 32)
        self.bn1rating = nn.BatchNorm1d(32)
        self.fc2rating = nn.Linear(32, 16)
        self.bn2rating = nn.BatchNorm1d(16)
        
        
        
        self.fc1res = nn.Linear(32*1 + 16*1, 32)
        self.bn1res = nn.BatchNorm1d(32)
        self.fc2res = nn.Linear(32, 1)
        
        self.do = nn.Dropout(0.2)
    
    def predict(self, figs_1, figs_2, rating_1, rating_2):
        result = self.forward(figs_1, figs_2, rating_1, rating_2)
        result = F.sigmoid(result)
        return result
        
    def forward(self, figs_1, figs_2, rating_1, rating_2):
        figs_1 = self.fc1figs(figs_1)
        figs_1 = self.bn1figs(figs_1)
        figs_1 = F.relu(figs_1)
        #figs_1 = self.do(figs_1)
        
        figs_1 = self.fc2figs(figs_1)
        figs_1 = self.bn2figs(figs_1)
        figs_1 = F.relu(figs_1)
        #figs_1 = self.do(figs_1)
        
        figs_2 = self.fc1figs(figs_2)
        figs_2 = self.bn1figs(figs_2)
        figs_2 = F.relu(figs_2)
        #figs_2 = self.do(figs_2)
        
        figs_2 = self.fc2figs(figs_2)
        figs_2 = self.bn2figs(figs_2)
        figs_2 = F.relu(figs_2)
        #figs_2 = self.do(figs_2)
        
        figs_diff = figs_1 - figs_2
        #figs_cross = torch.cross(figs_1, figs_2)
        
        
        rating_1 = self.fc1rating(rating_1)
        rating_1 = self.bn1rating(rating_1)
        rating_1 = F.relu(rating_1)
        #rating_1 = self.do(rating_1)
        
        rating_1 = self.fc2rating(rating_1)
        rating_1 = self.bn2rating(rating_1)
        rating_1 = F.relu(rating_1)
        #rating_1 = self.do(rating_1)
        
        
        rating_2 = self.fc1rating(rating_2)
        rating_2 = self.bn1rating(rating_2)
        rating_2 = F.relu(rating_2)
        #rating_2 = self.do(rating_2)
        
        rating_2 = self.fc2rating(rating_2)
        rating_2 = self.bn2rating(rating_2)
        rating_2 = F.relu(rating_2)
        #rating_2 = self.do(rating_2)
        
        rating_diff = rating_1 - rating_2
        #rating_cross = torch.cross(rating_1, rating_2)
        
        features = torch.cat((figs_diff, rating_diff), dim=-1)
        features = self.fc1res(features)
        features = self.bn1res(features)
        features = F.relu(features)
        #features = self.do(features)
        
        result = self.fc2res(features)
        return result

In [19]:
batch_size = 64
train_dataset = TrainDataset(fig_1_train, fig_2_train, rating_1_train, rating_2_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TrainDataset(fig_1_val, fig_2_val, rating_1_val, rating_2_val, y_val)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = TestDataset(fig_1_test, fig_2_test, rating_1_test, rating_2_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [20]:
rating_2_train.shape

(2906404, 17)

In [21]:
model = NN()
if device == 'cuda':
    model = model.cuda()

In [22]:
epochs = 40
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7, last_epoch=-1)

In [23]:
best_valid_loss = float('inf')
for epoch in range(1, epochs + 1):
    train_loss = round(train_epoch(train_dataloader, optimizer, criterion), 7)
    valid_loss = round(val_epoch(valid_dataloader, criterion), 7)
    if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), './data/models/result_same_weights_no_dropout.pt')

    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    scheduler.step()

45413it [04:48, 157.59it/s]
11354it [00:20, 549.69it/s]
0it [00:00, ?it/s]

Epoch: 1 
 Train Loss 0.5119795  Val loss 0.5027951:


45413it [04:48, 157.20it/s]
11354it [00:20, 554.42it/s]
0it [00:00, ?it/s]

Epoch: 2 
 Train Loss 0.4964456  Val loss 0.4888294:


45413it [04:49, 157.13it/s]
11354it [00:20, 561.13it/s]
0it [00:00, ?it/s]

...

Epoch: 38 
 Train Loss 0.4729987  Val loss 0.4710159:


45413it [05:14, 144.59it/s]
11354it [00:21, 522.67it/s]
0it [00:00, ?it/s]

Epoch: 39 
 Train Loss 0.4729019  Val loss 0.4709228:


45413it [05:14, 144.45it/s]
11354it [00:22, 500.83it/s]

Epoch: 40 
 Train Loss 0.4729063  Val loss 0.4702604:


In [24]:
model.load_state_dict(torch.load('./data/models/result_same_weights_no_dropout.pt'))

<All keys matched successfully>

In [25]:
round(val_epoch(valid_dataloader, criterion), 7)

11354it [00:21, 517.75it/s]


0.4694222

In [26]:
res = predict(test_dataloader)

0it [00:00, ?it/s]/home/mpopov/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
14192it [00:24, 567.73it/s]


In [27]:
res = [item[0] for item in res]

In [28]:
df_test['target'] = res

In [29]:
df_test[['id','target']].to_csv('result_same_weights_no_dropout.csv', index=False)

In [31]:
!head  result_same_weights_no_dropout.csv

id,target
0,0.44777318835258484
1,0.01042748149484396
2,0.4118897020816803
3,0.15768353641033173
4,0.19546987116336823
5,0.15544505417346954
6,0.7730392217636108
7,0.3866017162799835
8,0.8142403960227966
